<a href="https://colab.research.google.com/github/SHUBHAMDw/First/blob/master/YouTube_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install Dependencies
!pip install openai "elevenlabs>=1.0.0" requests moviepy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.1/740.1 kB 8.3 MB/s eta 0:00:00


In [2]:
# Cell 2: Create the news source file
%%writefile news_sources.csv
topic,url,keywords
"AI Chip Race","https://www.reuters.com/technology/nvidia-partners-indias-reliance-tata-ai-infrastructure-2023-09-08/","technology, AI, computer chip"
"New Space Telescope","https://www.example.com/some-space-article","space, galaxy, stars"

Writing news_sources.csv


In [5]:
# Cell 3: The Full Pipeline Code

import os
import csv
import requests
from google.colab import userdata  # The Colab way to get secrets
from openai import OpenAI
from elevenlabs import Voice, VoiceSettings
from elevenlabs.client import ElevenLabs
from moviepy.editor import *

# --- 1. CONFIGURATION & SETUP ---
# Load API keys from Colab Secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
ELEVENLABS_API_KEY = userdata.get('ELEVENLABS_API_KEY')
PEXELS_API_KEY = userdata.get('PEXELS_API_KEY')

# ElevenLabs Voice ID (Replace with your chosen voice ID from the ElevenLabs website)
ELEVENLABS_VOICE_ID = "21m00Tcm4TlvDq8ikWAM" # Example: Rachel's voice

# Initialize API clients
openai_client = OpenAI(api_key=OPENAI_API_KEY)
elevenlabs_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

# --- 2. PIPELINE FUNCTIONS (Identical to the previous script) ---

def get_news_from_source(file_path='news_sources.csv'):
    with open(file_path, 'r') as f:
        reader = csv.DictReader(f)
        return next(reader)

def generate_news_script(topic, url):
    print(f"-> Generating script for topic: {topic}")
    prompt = f"""
    You are a professional news anchor. Write a concise and engaging 150-word news script
    about the topic "{topic}". The information is based on the article at this URL: {url}.
    Start with a strong hook and maintain a professional, clear tone.
    Do not include any intro or sign-off like "Hello and welcome" or "Thanks for watching".
    Just provide the script body.
    """
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    script = response.choices[0].message.content.strip()
    print(f"-> Script generated successfully.")
    return script

def generate_anchor_audio(script_text, output_path="anchor_audio.mp3"):
    print("-> Generating anchor audio...")
    audio = elevenlabs_client.generate(
        text=script_text,
        voice=Voice(voice_id=ELEVENLABS_VOICE_ID),
        model="eleven_multilingual_v2"
    )
    with open(output_path, "wb") as f:
        f.write(audio)
    print(f"-> Audio saved to {output_path}")
    return output_path

def find_and_download_broll(keywords, output_path="broll_video.mp4"):
    print(f"-> Searching for B-roll with keywords: {keywords}")
    headers = {"Authorization": PEXELS_API_KEY}
    query = keywords.split(',')[0].strip()
    url = f"https://api.pexels.com/videos/search?query={query}&per_page=1&orientation=landscape"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error: Failed to fetch video from Pexels. Response: {response.text}")
        return None
    data = response.json()
    if not data.get('videos'):
        print(f"Error: No videos found for '{query}'")
        return None
    video_files = data['videos'][0]['video_files']
    video_url = next((vid['link'] for vid in video_files if vid.get('quality') == 'hd'), None)
    if not video_url:
        print("Error: No suitable HD video link found.")
        return None
    print(f"-> Downloading B-roll from {video_url}")
    video_response = requests.get(video_url)
    with open(output_path, "wb") as f:
        f.write(video_response.content)
    print(f"-> B-roll saved to {output_path}")
    return output_path

def create_final_video(audio_path, video_path, output_path="final_video.mp4"):
    print("-> Assembling final video...")
    audio_clip = AudioFileClip(audio_path)
    video_clip = VideoFileClip(video_path)
    video_clip = video_clip.subclip(0, audio_clip.duration).fx(vfx.fadeout, 1)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        temp_audiofile='temp-audio.m4a',
        remove_temp=True
    )
    print(f"-> Final video saved to {output_path}")

# --- 3. MAIN EXECUTION BLOCK ---

def main():
    print("--- Automated Video Pipeline Started ---")
    news_item = get_news_from_source()
    script = generate_news_script(news_item['topic'], news_item['url'])
    audio_file = generate_anchor_audio(script)
    broll_video_file = find_and_download_broll(news_item['keywords'])
    if not broll_video_file:
        print("Halting pipeline: Could not get B-roll footage.")
        return
    create_final_video(audio_file, broll_video_file)
    print("--- Pipeline Complete ---")
    print("Find 'final_video.mp4' in the file browser on the left.")

# Run the pipeline!
main()

--- Automated Video Pipeline Started ---
-> Generating script for topic: AI Chip Race


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')